# About
This code is used to download information of a subhalo from IllustrisTNG and adapt the hdf5 file to the GADGET-4 snapshot format that can be read by the GLnemo2 program.

TNG project data acess:
https://www.tng-project.org/data/

TNG project data specifications:
https://www.tng-project.org/data/docs/specifications/

# 1º downloading the necessary information from Illustris

In [1]:
#libs
import requests

#Halo parameters
simulation_name = "TNG50-1"
halo_id = 117255
snapshot = 99

#Base URL
base_url = f"http://www.tng-project.org/api/{simulation_name}"

#token API
headers = {"api-key":"[your_api_key]"}  # replace "your_api_key" with the API key obtained when registering on the project site

In [2]:
#Make a request for subhalo information
url = f"{base_url}/snapshots/{snapshot}/subhalos/{halo_id}"
response = requests.get(url, headers=headers)

if response.status_code != 200:
    print(f"Request failed with status {response.status_code}")
else:
    subhalo_data = response.json()

In [9]:
#Request to download the cut of the subhalo
cutout_request = {"gas":"Coordinates,Density,ElectronAbundance,InternalEnergy,Masses,GFM_Metallicity,ParticleIDs,Velocities","dm":"Coordinates,ParticleIDs,Velocities","stars":"Coordinates,Masses,GFM_Metallicity,ParticleIDs,GFM_StellarFormationTime,Velocities"}  # modify as necessary
url = f"{base_url}/snapshots/{snapshot}/subhalos/{halo_id}/cutout.hdf5"
response = requests.get(url, headers=headers, params=cutout_request)

#Final check if the order was successful
if response.status_code != 200:
    print(f"Request failed with status {response.status_code}")
else:
    with open(f"halo_{halo_id}_cutout.hdf5", "wb") as file:
        file.write(response.content)
    print(f"Download the halo file_{halo_id}_cutout.hdf5 completed successfully.")

Download the halo file_117255_cutout.hdf5 completed successfully.


# 2º Creating a snapshot with the downloaded information that can be read by GLnemo2

In [10]:
#libs
import h5py
import numpy as np

# Opens the 'si' and 'sh' files in read-only mode. The 'sh' is only used to get the format of the header information.
with h5py.File(f'halo_{halo_id}_cutout.hdf5', 'r') as si, h5py.File('snapshot_header.hdf5', 'r') as sh:
    
    # Creates a copy of the 'sh' file called 'sh_copy.hdf5'
    with h5py.File(f'halo_{halo_id}_correction.hdf5', 'w') as sh_copy:
        
        # Copy group by group from file 'sh' to 'sh_copy'
        for item in sh:
            sh.copy(item, sh_copy)
        
        # Inicializa um array para armazenar o número de partículas de cada tipo
        numpart_file = np.zeros(6, dtype=np.uint64)
        
        # Initializes an array to store the number of particles of each type
        for part_type in ['PartType0', 'PartType1', 'PartType2', 'PartType3', 'PartType4']:
            if part_type in si:
                # Copies the data from the 'si' file to 'sh_copy
                if part_type in sh_copy:
                    del sh_copy[part_type]  # Apaga o grupo existente
                si.copy(part_type, sh_copy)
                # Updates the number of particles for this type
                numpart_file[int(part_type[-1])] = len(si[part_type]['Coordinates'])
            else:
                # If the particle type does not exist in 'si', delete it from 'sh_copy' and set the number of particles to 0
                if part_type in sh_copy:
                    del sh_copy[part_type]
                numpart_file[int(part_type[-1])] = 0
        
        # Updates the header in 'sh_copy' with the correct number of particles
        sh_copy['Header'].attrs['NumPart_ThisFile'] = numpart_file
        sh_copy['Header'].attrs['NumPart_Total'] = numpart_file
        sh_copy['Header'].attrs['NumPart_Total_HighWord'] = numpart_file

        # Reads other attributes from the header of the 'si' file and updates them in the header of the 'sh_copy' file as necessary
        time_value = si['Header'].attrs['Time']
        sh_copy['Header'].attrs['Time'] = time_value
        
        redshift_value = si['Header'].attrs['Redshift']
        sh_copy['Header'].attrs['Redshift'] = redshift_value
        
        masstable_value = si['Header'].attrs['MassTable']
        sh_copy['Header'].attrs['MassTable'] =  masstable_value
        
        boxsize_value = si['Header'].attrs['BoxSize']
        sh_copy['Header'].attrs['BoxSize'] =  boxsize_value
        
        
        print(f"Correction of the file_{halo_id}_cutout.hdf5 completed successfully.")

Correction of the file_117255_cutout.hdf5 completed successfully.
